In [95]:
!pip install konlpy
import re
import pandas as pd
import numpy as np
from konlpy.tag import Okt
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [5]:
reviews = pd.read_csv('drive/My Drive/datasets/review_final.csv')
reviews = reviews.loc[:,['댓글내용']]

In [ ]:
reviews.sample(frac=1).reset_index(drop=True)

In [ ]:
!pip install chatspace

In [11]:
# 띄어쓰기 보정 오픈소스
from chatspace import ChatSpace

spacer = ChatSpace()
spacer.space(reviews['댓글내용'][0].replace('\n',''))

Loading JIT Compiled ChatSpace Model


'포장지 열자마자 유부초밥 냄새가 나서 자세히 봤더니 사진 처럼 국물이 고여있었어요 상품 포장상태가 안좋았던 건지 아니면 배송하시는 분이 던져서 터진 건지는 모르겠지만 별로네요 그냥 다 버려서 맛은 모르겠어요 맛 만족도 괜찮아요'

In [ ]:
test_data = []
for text in reviews['댓글내용']:
  test_data.append(spacer.space(text).replace('\n',''))
test_data

In [ ]:
test_data = pd.DataFrame(test_data, columns=['review'])
test_data

In [106]:
keyword_list = {'맛': ['맛','불맛', '감칠맛', '쓴맛', '향', '단맛', '신맛', '냄새', '끝맛', '짠맛', '꼬순내', '햄맛',
                      '햄향', '닭맛', '불향', '풋내', '조미료', '끝향', '쏘스맛', '닭내', '파맛', '존맛', '핵존맛', '텁텁한맛'],
                '가격':['가격', '금액', '값', '단가', '할인', '가성', '득템', '창렬', '혜자', '정가', '사기'],
                '양':['용량', '양은', '양도', '양으로', '개수', '인분', '분량', '물량', '사이즈', '수량','양이', '남았', '남아','많'],
                '품질':['품질', '상태', '아이스', '얼음', '비닐', '아이스팩', '뾱뾱', '뽁뽁이팩','드라이아이스', '아이스박스', '보냉', '상자',
                      '종이팩', '종이', '상온', '실온','상한','상했','썩',
                        '소재', '박스', '뽁뽁이', '비닐봉투', '밀봉', '보냉팩', '배송', '발송', '택배', '배달', '검사', '퀄리티', '퀄러티','환경',
                       '검증', '신뢰', '신선',  '유통기한', '유효기한', '싱싱', '상한', '부패', '냄새'],
                '간편':['간편', '간단', '초보', '방법', '쉬움', '어려움', '단시간', '출퇴근시간', '간편식', '전문식','편리', '손쉽', '혼밥', '꽝손', 
                      '불편', '똥손', '금손', '편의'],
                '외관':['외관', '육안', '디자인', '실물', '겉보기', '보고', '보임', '볼때', '볼 때', '보고', '맛보다', '봣', '눈으로', '사진찍기 좋은',
                      '먹음직한', '말쌀나는', '푸디용', '색감', '색깔', '색이', '모양새'],
                '웰빙':['웰빙', 'msg', '인스턴트', '건강식', '특수식품', '즉석식품', '건강', '다이어트', '다이어터', '헬스', '무첨가', '식단', '단백질',
                      '영양', '효과', '예방', '체중', '영양소', '탄수화물', '칼로리', '성분', '소화효소', '체중감량', '면역', '근육', '야맹증',
                      '질환', '보조', '미용', '운동', '에너지', '호르몬', '칼슘', '성인병', '갱년기', '성장', '비료', '노화', 
                      '치매', '효능', '염분', '두뇌', '불균형', '뇌', '알츠하이머', '코로나', '영양분', '소화']}

In [18]:
for keyword in keyword_list:
  test_data[keyword] = -1
test_data

,review,맛,가격,양,품질,간편,외관,웰빙
0,포장지 열자마자 유부초밥 냄새가 나서 자세히 봤더니 사진 처럼 국물이 고여있었어요 ...,-1,-1,-1,-1,-1,-1,-1
1,세 번째 시켜 먹었는데 깜짝 놀랐네요. 두부 넣어서 먹는 게 맛있어서 이번에도 시켰...,-1,-1,-1,-1,-1,-1,-1
2,안에 들어있는 유부팩 하나가 터져서 국물이 냉장고 안에 줄줄 흘렀어요.. 안에서 터...,-1,-1,-1,-1,-1,-1,-1
3,삼각은 싸서 입구를 막는데 사각은 토핑용인듯요 삼각보다 불편해요... ㅠㅠ,-1,-1,-1,-1,-1,-1,-1
4,맛이 좋아요 맛 만족도 예상보다 맛있어요,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...
87406,편리성 사용하기 편리해요 신선도 예상만큼 신선해요 맛 만족도 예상보다 맛있어요,-1,-1,-1,-1,-1,-1,-1
87407,편리성 그저 그래요 신선도 예상보다 신선해요 맛 만족도 괜찮아요,-1,-1,-1,-1,-1,-1,-1
87408,편리성 그저 그래요 신선도 예상만큼 신선해요 맛 만족도 괜찮아요,-1,-1,-1,-1,-1,-1,-1
87409,신선도 예상보다 신선해요 맛 만족도 예상보다 맛있어요,-1,-1,-1,-1,-1,-1,-1


In [86]:
# tokenizing
okt = Okt()
def tokenize(doc):
    # 한글 자음, 모음 제거
    doc = re.sub(pattern='([ㄱ-ㅎㅏ-ㅣ]+)', repl='', string=doc)
    # 특수기호 제거
    doc = re.sub(pattern='[^\w\s]', repl='', string=doc)
    # norm은 정규화, stem은 근어로 표시하기를 나타냄
    doc = okt.pos(doc, norm=True, stem=True)
    # 명사, 형용사, 부사, 동사 채택
    token = []
    for i in doc:
        if i[1] == 'Noun' or i[1] == 'Verb' or i[1] == 'Adverb' or i[1] == 'Adjective':
          token.append(i)

    return ['/'.join(t) for t in token]

In [96]:
def vectorize(sentence, encoder):
  l = []
  token = tokenize(sentence)
  l.append(token)
  result = encoder.texts_to_sequences(l)
  return pad_sequences(result, 100)

In [93]:
# 인코더 기존 학습된 모델 사용
import pickle
with open('drive/My Drive/datasets/encoder_v2.pickle', 'rb') as handle:
    encoder = pickle.load(handle)

In [107]:
def inject_keyword(text, window):
  # 결과값
  result = {'맛':'', '가격': '', '양':'', '품질':'', '간편':'', '외관':'', '웰빙':''}
  sentences = text.split('.') # 문장 분리

  # 맛 ~ 웰빙까지
  for key in keyword_list.keys():
    keys = keyword_list[key] # 키워드 + 연관 키워드 리스트
    injected = '' # 추출할 문장

    for k in keys: 
      for sentence in sentences:
        words = sentence.split(' ') # 각 문장 띄어쓰기 분리
        for word in words:
          if k in word:
            index = words.index(word)
            # 키워드에 해당하는 단어의 인덱스를 기준으로 앞 뒤 몇 글자씩 잘라내서 추출
            # window 파라미터로 몇 글자를 잘라낼 것인지 설정
            # 문맥 상 키워드의 이전 단어들 보다 이후에 등장하는 단어들을 더 많이 추출
            for i in range(window-2):
              if index - (window-2 - i) >= 0:
                injected += words[index - (window-2 - i)]
                injected += ' '
            for i in range(window+2):
              if index + i < len(words):
                injected += words[index + i]
                injected += ' '    
            break
                 
      result[key] = injected
      
  return result

In [121]:
inject_keyword(test_data['review'][0], 7)

{'가격': '',
 '간편': '',
 '맛': ' 별로네요 그냥 다 버려서 맛은 모르겠어요 맛 만족도 괜찮아요 포장지 열자마자 유부초밥 냄새가 나서 자세히 봤더니 사진 처럼 국물이 고여있었어요 상품 ',
 '양': '',
 '외관': '',
 '웰빙': '',
 '품질': '사진 처럼 국물이 고여있었어요 상품 포장상태가 안좋았던 건지 아니면 배송하시는 분이 던져서 터진 건지는 상품 포장상태가 안좋았던 건지 아니면 배송하시는 분이 던져서 터진 건지는 모르겠지만  별로네요 그냥 포장지 열자마자 유부초밥 냄새가 나서 자세히 봤더니 사진 처럼 국물이 고여있었어요 상품 '}

In [119]:
# 테스트용 3000개 데이터 사용
samples = test_data[:3000]
# 사전 학습된 LSTM 모델 사용
model = load_model('drive/My Drive/datasets/LSTM_binary.h5')
for sample in samples['review']:
  inject_result = inject_keyword(sample, 3)
  for key in keyword_list.keys():
    inject_sentence = inject_result[key]
    # 해당 키워드가 없으면 점수는 -1
    if inject_sentence == '':
      samples.loc[sample, key] = -1
    else:
      samples.loc[sample, key] = model.predict(vectorize(inject_sentence, encoder))[0][0] * 100

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [ ]:
samples[samples['맛'] != -1]

In [ ]:
samples[samples['가격']!= -1]

비고 
 * 아직까지 정확도에 한계가 있다. 키워드 추출 로직도 보완 필요.
 * 키워드 연관 단어 사전 역시 보완 필요
 * POC 나 프로토타입 때 보여주기 용으로는 나쁘지 않은듯.
 * 계산 시간 비용이 많이 든다. 3000개 학습 시 10분 정도, 9만 개 학습 시 5시간